In [10]:
import requests

# instruction = """
# کمترین قیمت ست سطل و جای دستمال لالیپاپ مدل رنگین کمان چنده؟
# """
# instruction = "لطفاً دراور چهار کشو (کد D14) را برای من تهیه کنید."
# instruction = "عرض پارچه تریکو جودون 1/30 لاکرا گردباف نوریس به رنگ زرد طلایی چقدر است؟"
instruction = "کمترین قیمت در این پایه برای گیاه طبیعی بلک گلد بنسای نارگل کد ۰۱۰۸ چقدر است؟"
# instruction = "یک لپ تاپ مناسب بهم پیشنهاد کن."
# instruction = """
# چند تا کالا پیدا کن که برای فروشگاه های متفاوتی هستند و بهم معرفی کن.
# """
instruction = """
"کدام یک از این ماگ‌های خرید ماگ-لیوان هندوانه فانتزی و کارتونی کد 1375 یا ماگ لته خوری سرامیکی با زیره کد 741 دارای سبک کارتونی و فانتزی بوده و برای کودکان یا نوجوانان مناسب‌تر است؟"
"""
# instruction = """
# قیمت متوسط خردکن سیلور کرست مدل NF-1923 تیغه تیتانیومی در شهر قم چقدر است؟
# """

url = "https://behnam-torob-ai-shopping-assistant.darkube.app/chat"
data = {
    "chat_id": "embedding_qdrant_initialization-CODE554242fwrh2j42hj41589",
    "messages": [{"type": "text", "content": instruction }]
}

# Disable system proxy
resp = requests.post(url, json=data, proxies={"http": None, "https": None})

In [11]:
resp.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)